In [ ]:
# | default_exp social_image_generator


In [ ]:
# | export

from typing import *
from pathlib import Path
import re
import os
import asyncio
import shutil
from tempfile import TemporaryDirectory
from enum import Enum

import openai
import typer
from playwright.async_api import async_playwright
from ruamel.yaml import YAML

from nbdev_mkdocs._helpers.utils import (
    set_cwd,
    get_value_from_config,
    is_local_path,
    add_counter_suffix_to_filename,
)
from nbdev_mkdocs._package_data import get_root_data_path


In [ ]:
import pytest


In [ ]:
# | export


def _generate_ai_image(prompt: str) -> str:
    """Generate an image for social card using the OpenAI Image API.

    Args:
        prompt: The prompt to use for generating the image.

    Returns:
        The URL of the generated image.
    """
    response = openai.Image.create(prompt=prompt, n=1, size="512x512")
    image_url = response["data"][0]["url"]
    return image_url


In [ ]:
prompt = "cute animal, browsing computer, high tech, flat, digital art"

with pytest.raises(Exception) as e:
    _generate_ai_image(prompt=prompt)

e.value


InvalidRequestError(message='Billing hard limit has been reached', param=None, code='billing_hard_limit_reached', http_status=400, request_id=None)

In [ ]:
# | export


def _generate_html_str(root_path: str, image_url: str) -> str:
    """Generate html string for the social card

    Args:
        root_path: The root path of the project.
        image_url: The image URL to be included in the HTML.
    """

    with set_cwd(root_path):

        _custom_social_image_template_path = (
            get_root_data_path() / "custom-social-image-template.html"
        )

        with open(_custom_social_image_template_path, "r") as f:
            _html_template = f.read()

        user_name = get_value_from_config(root_path, "user")
        project_name = get_value_from_config(root_path, "repo")
        project_description = get_value_from_config(root_path, "description")

        image_url = Path(image_url).name if is_local_path(image_url) else image_url

        d = dict(
            fill_in_user_name=user_name,
            fill_in_project_name=project_name,
            fill_in_project_description=project_description,
            fill_in_image_url=image_url,
        )

        for k, v in d.items():
            _html_template = _html_template.replace(k, v)

        return _html_template


In [ ]:
with TemporaryDirectory() as d:

    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    user_name = get_value_from_config(d, "user")
    project_name = get_value_from_config(d, "repo")
    project_description = get_value_from_config(d, "description")

    image_url = "https://sample-image.png"
    actual = _generate_html_str(d, image_url)
    print(actual)

    assert f"{user_name}/" in actual
    assert project_name in actual
    assert image_url in actual


<!DOCTYPE html>
<html>
    <head>
        <link rel="preload" href='https://fonts.googleapis.com/css?family=Source Sans Pro' rel='stylesheet'/>
        <style>
            body {
                margin: 0px;
                width: 1200px;
                height: 630px;
                font-family: 'Source Sans Pro', sans-serif;
            }
            #container {
                display: flex;
                height: 630px;
            }
            #container div:first-child{
                flex-basis: 704px;
                padding-top: 40px;
            }
            #container div:first-child h1{
                font-size: 80px;
                margin: 74px 0px 64px 60px;
                color: #2E363D;
            }
            #container div:first-child h1 span{
                font-weight: 100;
            }
            #container div:first-child p {
                font-size: 25px;
                color: #6E7681;
                margin: 64px 0px 64px 64px;
                p

In [ ]:
with TemporaryDirectory() as d:

    shutil.copyfile(Path("..") / "settings.ini", Path(d) / "settings.ini")

    user_name = get_value_from_config(d, "user")
    project_name = get_value_from_config(d, "repo")
    project_description = get_value_from_config(d, "description")

    image_url = Path(d) / "sample-image.png"
    actual = _generate_html_str(d, image_url)
    print(actual)

    assert f"{user_name}/" in actual
    assert project_name in actual
    assert str(image_url) not in actual
    assert "sample-image.png" in actual


<!DOCTYPE html>
<html>
    <head>
        <link rel="preload" href='https://fonts.googleapis.com/css?family=Source Sans Pro' rel='stylesheet'/>
        <style>
            body {
                margin: 0px;
                width: 1200px;
                height: 630px;
                font-family: 'Source Sans Pro', sans-serif;
            }
            #container {
                display: flex;
                height: 630px;
            }
            #container div:first-child{
                flex-basis: 704px;
                padding-top: 40px;
            }
            #container div:first-child h1{
                font-size: 80px;
                margin: 74px 0px 64px 60px;
                color: #2E363D;
            }
            #container div:first-child h1 span{
                font-weight: 100;
            }
            #container div:first-child p {
                font-size: 25px;
                color: #6E7681;
                margin: 64px 0px 64px 64px;
                p

In [ ]:
# | export


async def _capture_and_save_screenshot(src_path: str, dst_path: str):
    """Capture screenshot of an HTML file from source directory and save the
    output in destination directory

    Args:
        src_path: The source path of the HTML file that will be used to generate the PNG image.
        dst_path: The destination path where the generated screenshot image will be saved.
    """
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch()

    try:
        page = await browser.new_page()

        html_path = Path(src_path) / "social_image.html"
        await page.goto(f"file://{str(html_path.resolve())}")

        output_path = (
            Path(dst_path) / "mkdocs" / "docs_overrides" / "images" / "social_image.png"
        )

        if output_path.exists():
            add_counter_suffix_to_filename(output_path)

        await page.locator("#container").screenshot(path=str(output_path.resolve()))

        typer.echo(f"Social share image generated and saved at: '{output_path}'")

    finally:
        await browser.close()


In [ ]:
with TemporaryDirectory() as d:

    html_path = Path(d) / "social_image.html"
    with open(html_path, "w") as f:
        f.write("<html><body><div id = 'container'><p>This is a sample text</p></div></body></html>")

    png_path = Path(d) / "dst_path" / "mkdocs" / "docs_overrides" / "images"
    png_path.mkdir(parents=True)

    dst_path = Path(d) / "dst_path"
    await _capture_and_save_screenshot(d, dst_path)

    assert (png_path / "social_image.png").exists()
    
    await _capture_and_save_screenshot(d, dst_path)
    
    assert (png_path / "social_image.png").exists()
    assert (png_path / "social_image.1.png").exists()
    
    await _capture_and_save_screenshot(d, dst_path)
    
    assert (png_path / "social_image.png").exists()
    assert (png_path / "social_image.1.png").exists()
    assert (png_path / "social_image.2.png").exists()
    
    (png_path / "social_image.1.png").unlink()
    await _capture_and_save_screenshot(d, dst_path)
    
    assert (png_path / "social_image.png").exists()
    assert (png_path / "social_image.2.png").exists()
    assert (png_path / "social_image.3.png").exists()
    
    !ls -la {png_path}


Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2r075q1i/dst_path/mkdocs/docs_overrides/images/social_image.png'
Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2r075q1i/dst_path/mkdocs/docs_overrides/images/social_image.png'
Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2r075q1i/dst_path/mkdocs/docs_overrides/images/social_image.png'
Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp2r075q1i/dst_path/mkdocs/docs_overrides/images/social_image.png'
total 24
drwxr-xr-x  5 harishm  staff   160 Dec 13 17:05 .
drwxr-xr-x  3 harishm  staff    96 Dec 13 17:05 ..
-rw-r--r--  1 harishm  staff  2832 Dec 13 17:05 social_image.2.png
-rw-r--r--  1 harishm  staff  2832 Dec 13 17:05 social_image.3.png
-rw-r--r--  1 harishm  staff  2832 Dec 13 17:05 social_image.png


In [ ]:
# | export


async def _create_social_image(root_path: str, image_url: str):
    """Create social image for the project

    Args:
        root_path: The root path of the project.
        image_url: The image URL to be included in the social image.
    """
    html_str = _generate_html_str(root_path, image_url)

    with TemporaryDirectory() as d:

        html_path = Path(d) / "social_image.html"
        with open(html_path, "w") as f:
            f.write(html_str)

        if is_local_path(image_url):
            shutil.copyfile(Path(image_url), Path(d) / Path(image_url).name)

        await _capture_and_save_screenshot(d, root_path)


In [ ]:
with TemporaryDirectory() as d:

    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(parents=True)

    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images" / "default_social_logo.png",
        Path(d) / "default_social_logo.png",
    )

    image_url = (Path(d) / "default_social_logo.png").resolve()
    await _create_social_image(root_path=d, image_url=image_url)

    png_path = mkdocs_dir_path / "social_image.png"
    assert png_path.exists()


Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4yj4z4zv/mkdocs/docs_overrides/images/social_image.png'


In [ ]:
# | export


def _update_social_image_in_mkdocs_yml(root_path: str, image_url: str):
    """Update social image link in mkdocs yml file

    Args:
        root_path: The root path of the project.
        image_url: The image URL to update in the mkdocs yml file.
    """

    image_url = (
        "overrides/images/social_image.png" if is_local_path(image_url) else image_url
    )

    yaml = YAML()
    mkdocs_yml_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
    config = yaml.load(mkdocs_yml_path)
    config["extra"]["social_image"] = image_url
    yaml.dump(config, mkdocs_yml_path)


In [ ]:
with TemporaryDirectory() as d:

    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(parents=True)

    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images" / "default_social_logo.png",
        Path(d) / "default_social_logo.png",
    )

    image_url = (Path(d) / "default_social_logo.png").resolve()
    await _create_social_image(root_path=d, image_url=image_url)

    png_path = mkdocs_dir_path / "social_image.png"
    assert png_path.exists()

    mkdocs_yml_path = Path(d) / "mkdocs"
    shutil.copyfile(
        Path("..") / "mkdocs" / "mkdocs.yml", (mkdocs_yml_path / "mkdocs.yml")
    )

    image_url = "https://my-random-domain/sample.png"
    _update_social_image_in_mkdocs_yml(d, image_url)

    yaml = YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config["extra"]["social_image"])
    assert config["extra"]["social_image"] == image_url, config["extra"]["social_image"]

    image_url = Path(d) / "sample-image.png"
    _update_social_image_in_mkdocs_yml(d, image_url)

    yaml = YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config["extra"]["social_image"])
    assert (
        config["extra"]["social_image"] == "overrides/images/social_image.png"
    ), config["extra"]["social_image"]


Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpryyi5rbm/mkdocs/docs_overrides/images/social_image.png'
https://my-random-domain/sample.png
overrides/images/social_image.png


In [ ]:
# | export


def _update_social_image_in_site_overrides(root_path: str, image_url: str):
    """Update social image link in site_overrides HTML template

    Args:
        root_path: The root path of the project.
        image_url: The image URL to update in the site_overrides HTML template.
    """

    _replace_str = (
        'page.canonical_url ~ "" ~ config.extra.social_image '
        if is_local_path(image_url)
        else "config.extra.social_image "
    )

    with set_cwd(root_path):
        site_overrides_path = (
            Path(root_path) / "mkdocs" / "site_overrides" / "main.html"
        )
        if not site_overrides_path.exists():
            typer.secho(
                f"Unexpected error: path {site_overrides_path.resolve()} does not exists!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(code=1)

        with open(site_overrides_path, "r") as f:
            _new_text = f.read()
            _pattern = re.compile(r".*?{%.*?image_url = (.*)%}")
            _match = re.search(_pattern, _new_text)
            _new_text = _new_text.replace(_match.group(1), _replace_str)  # type: ignore

        with open(site_overrides_path, "w") as f:
            f.write(_new_text)


In [ ]:
with TemporaryDirectory() as d:

    site_overrides_path = Path(d) / "mkdocs" / "site_overrides"
    site_overrides_path.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(
        Path("..") / "mkdocs" / "site_overrides" / "main.html",
        (site_overrides_path / "main.html"),
    )

    image_url = Path(d) / "random-image.png"
    _update_social_image_in_site_overrides(d, image_url)

    with open((site_overrides_path / "main.html")) as f:
        actual = f.read()

    print(actual)

    assert (
        '{% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}'
        in actual
    ), actual


{% extends "base.html" %}

{% block extrahead %}
  {% set title = config.site_name %}
  {% if page and page.meta and page.meta.title %}
    {% set title = title ~ " - " ~ page.meta.title %}
  {% elif page and page.title and not page.is_homepage %}
    {% set title = title ~ " - " ~ page.title | striptags %}
  {% endif %}
  {% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}
  <meta property="og:type" content="website" />
  <meta property="og:title" content="{{ title }}" />
  <meta property="og:description" content="{{ config.site_description }}" />
  <meta property="og:url" content="{{ page.canonical_url }}" />
  <meta property="og:image" content="{{ image_url }}" />
  <meta property="og:image:type" content="image/png" />
  <meta property="og:image:width" content="1200" />
  <meta property="og:image:height" content="630" />

  <meta name="twitter:card" content="summary_large_image" />
  <meta name="twitter:title" content="{{ title }}" />
  <meta name="twitter:desc

In [ ]:
with TemporaryDirectory() as d:

    site_overrides_path = Path(d) / "mkdocs" / "site_overrides"
    site_overrides_path.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(
        Path("..") / "mkdocs" / "site_overrides" / "main.html",
        (site_overrides_path / "main.html"),
    )

    image_url = "https://random-domain/random-image.png"
    _update_social_image_in_site_overrides(d, image_url)

    with open((site_overrides_path / "main.html")) as f:
        actual = f.read()

    print(actual)

    assert "{% set image_url = config.extra.social_image %}" in actual, actual


{% extends "base.html" %}

{% block extrahead %}
  {% set title = config.site_name %}
  {% if page and page.meta and page.meta.title %}
    {% set title = title ~ " - " ~ page.meta.title %}
  {% elif page and page.title and not page.is_homepage %}
    {% set title = title ~ " - " ~ page.title | striptags %}
  {% endif %}
  {% set image_url = config.extra.social_image %}
  <meta property="og:type" content="website" />
  <meta property="og:title" content="{{ title }}" />
  <meta property="og:description" content="{{ config.site_description }}" />
  <meta property="og:url" content="{{ page.canonical_url }}" />
  <meta property="og:image" content="{{ image_url }}" />
  <meta property="og:image:type" content="image/png" />
  <meta property="og:image:width" content="1200" />
  <meta property="og:image:height" content="630" />

  <meta name="twitter:card" content="summary_large_image" />
  <meta name="twitter:title" content="{{ title }}" />
  <meta name="twitter:description" content="{{ confi

In [ ]:
# | export


class _IMG_Generator(str, Enum):
    file = "file"
    dall_e = "dall_e"


def _generate_image_url(
    root_path: str, generator: str, prompt: str, image_path: Optional[str] = None
) -> str:
    """Generate image url based on the generator

    Args:
        root_path: The root path of the project.
        generator: Generator to use to create the social image. Valid options are: 'file' and 'dall_e'.
        prompt: The prompt to use for generating the image.
        image_path: Image file path to use in the social share image. If None, then the default image will be used.

    Returns:
        The image url
    """
    if generator not in _IMG_Generator._member_names_:
        raise ValueError(
            "Invalid Option for generator. Valid options are: 'file' and 'dall_e'"
        )

    elif generator == _IMG_Generator.dall_e.value:
        image_url = _generate_ai_image(prompt=prompt)

    else:
        with set_cwd(root_path):
            if image_path is not None:
                _image_path = Path(
                    os.path.normpath(Path(root_path).joinpath(image_path))
                ).resolve()

                if not _image_path.exists():
                    typer.secho(
                        f"Unexpected error: path {_image_path.resolve()} does not exists!",
                        err=True,
                        fg=typer.colors.RED,
                    )
                    raise typer.Exit(code=1)

                image_url = str(_image_path)

            else:
                image_url = str(
                    (
                        Path(root_path)
                        / "mkdocs"
                        / "docs_overrides"
                        / "images"
                        / "default_social_logo.png"
                    ).resolve()
                )

    return image_url


In [ ]:
with TemporaryDirectory() as d:
    with pytest.raises(ValueError) as e:
        generator = "random_name"
        image_path = None
        prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"
        _generate_image_url(d, generator, prompt)

    print(e.value)


Invalid Option for generator. Valid options are: 'file' and 'dall_e'


In [ ]:
with TemporaryDirectory() as d:
    prompt = "The quick brown fox jumps over a lazy dog"
    generator = "dall_e"
    prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"

    with pytest.raises(Exception) as e:
        _generate_image_url(root_path=d, generator=generator, prompt=prompt)
    print(e.value)


Billing hard limit has been reached


In [ ]:
with TemporaryDirectory() as d:
    generator = "file"
    image_path = "invalid_image_path.png"
    prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"
    with pytest.raises(typer.Exit) as e:
        _generate_image_url(
            root_path=d, generator=generator, prompt=prompt, image_path=image_path
        )

    print(e.value)


Unexpected error: path /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpm_m_jhuy/invalid_image_path.png does not exists!


In [ ]:
with TemporaryDirectory() as d:

    prompt = "The quick brown fox jumps over a lazy dog"
    generator = "file"
    image_path = "./valid_image_path.png"
    prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"
    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images" / "compass-outline.png",
        Path(d) / "valid_image_path.png",
    )

    image_url = _generate_image_url(
        root_path=d, generator=generator, prompt=prompt, image_path=image_path
    )

    print(image_url)

    assert image_url == str(Path(d).resolve() / image_path), str(
        Path(d).resolve() / image_path
    )


/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp1ian3_wk/valid_image_path.png


In [ ]:
with TemporaryDirectory() as d:

    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images" / "compass-outline.png",
        Path(d) / "valid_image_path.png",
    )

    prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"
    generator = "file"

    d_tmp = Path(d) / "tmp"
    d_tmp.mkdir(parents=True)

    image_path = "../valid_image_path.png"

    image_url = _generate_image_url(
        root_path=d_tmp, generator=generator, prompt=prompt, image_path=image_path
    )

    print(image_url)

    assert image_url == str(Path(d).resolve() / "valid_image_path.png"), str(
        Path(d).resolve() / "valid_image_path.png"
    )


/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpfyxz0fvb/valid_image_path.png


In [ ]:
with TemporaryDirectory() as d:
    generator = "file"
    prompt = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render"
    image_url = _generate_image_url(root_path=d, generator=generator, prompt=prompt)
    print(image_url)
    assert image_url == str(
        (
            Path(d) / "mkdocs" / "docs_overrides" / "images" / "default_social_logo.png"
        ).resolve()
    ), image_url


/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpe5sxracs/mkdocs/docs_overrides/images/default_social_logo.png


In [ ]:
# | export


async def generate_social_image(
    root_path: str,
    generator: str = "file",
    prompt: str = "Cute animal wearing hoodie sitting in high chair in purple room, browsing computer, 3d render",
    image_path: Optional[str] = None,
):
    """Generate a custom image for social card using the OpenAI Image API.

    Args:
        root_path: The root path of the project.
        generator: Generator to use to create the social image. Valid options are: 'file' and 'dall_e'.
        prompt: The prompt to use for generating the image.
        image_path: Image file path to use in the social share image. If None, then the default image will be used.
    """

    image_url = _generate_image_url(root_path, generator, prompt, image_path)

    await _create_social_image(root_path, image_url)

    _update_social_image_in_mkdocs_yml(root_path, image_url)

    _update_social_image_in_site_overrides(root_path, image_url)


In [ ]:
with TemporaryDirectory() as d:

    mkdocs_dir_path = Path(d) / "mkdocs" / "docs_overrides" / "images"
    mkdocs_dir_path.mkdir(exist_ok=True, parents=True)

    mkdocs_yml_path = Path(d) / "mkdocs"
    shutil.copyfile(
        Path("..") / "mkdocs" / "mkdocs.yml", (mkdocs_yml_path / "mkdocs.yml")
    )

    site_overrides_path = Path(d) / "mkdocs" / "site_overrides"
    site_overrides_path.mkdir(exist_ok=True, parents=True)
    shutil.copyfile(
        Path("..") / "mkdocs" / "site_overrides" / "main.html",
        (site_overrides_path / "main.html"),
    )

    shutil.copyfile(
        Path("..") / "mkdocs" / "docs_overrides" / "images" / "default_social_logo.png",
        mkdocs_dir_path / "default_social_logo.png",
    )

    prompt = "The quick brown fox jumps over a lazy dog"
    await generate_social_image(root_path=d, prompt=prompt)

    png_path = mkdocs_dir_path / "social_image.png"

    assert png_path.exists()

    yaml = YAML()
    config = yaml.load((Path(d) / "mkdocs/mkdocs.yml"))
    print(config["extra"]["social_image"])
    assert (
        config["extra"]["social_image"] == "overrides/images/social_image.png"
    ), config["extra"]["social_image"]

    with open((site_overrides_path / "main.html")) as f:
        actual = f.read()

    print(actual)

    assert (
        '{% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}'
        in actual
    ), actual


Social share image generated and saved at: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpiho33ri8/mkdocs/docs_overrides/images/social_image.png'
overrides/images/social_image.png
{% extends "base.html" %}

{% block extrahead %}
  {% set title = config.site_name %}
  {% if page and page.meta and page.meta.title %}
    {% set title = title ~ " - " ~ page.meta.title %}
  {% elif page and page.title and not page.is_homepage %}
    {% set title = title ~ " - " ~ page.title | striptags %}
  {% endif %}
  {% set image_url = page.canonical_url ~ "" ~ config.extra.social_image %}
  <meta property="og:type" content="website" />
  <meta property="og:title" content="{{ title }}" />
  <meta property="og:description" content="{{ config.site_description }}" />
  <meta property="og:url" content="{{ page.canonical_url }}" />
  <meta property="og:image" content="{{ image_url }}" />
  <meta property="og:image:type" content="image/png" />
  <meta property="og:image:width" content="1200" />
  <meta 